In [ ]:
import boto3
import logging
import json
import time

# Configure logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Initialize EC2 client
ec2_client = boto3.client('ec2')

def lambda_handler(event, context):
    """
    Lambda handler to perform various EC2 operations based on the event's action.
    """
    start_time = time.time()  # Start tracking execution time
    logger.info(f"Received event: {json.dumps(event)}")  # Log the received event

    # Extract the action
    action = event.get('detail', {}).get('eventName')

    # Validate action input
    if not action:
        logger.warning("Missing 'eventName' in event detail.")
        return {"statusCode": 400, "body": "Missing 'eventName' in event detail."}

    try:
        # Handle StopInstances action
        if action == "StopInstances":
            response = ec2_client.describe_instances(
                Filters=[{'Name': 'instance-state-name', 'Values': ['running']}]
            )
            instance_ids = [
                instance['InstanceId']
                for reservation in response['Reservations']
                for instance in reservation['Instances']
            ]
            if not instance_ids:
                logger.info("No running instances found to stop.")
                return {"statusCode": 200, "body": "No running instances to stop."}

            logger.info(f"Stopping instances: {instance_ids}")
            stop_response = ec2_client.stop_instances(InstanceIds=instance_ids)
            logger.info(f"Stop response: {json.dumps(stop_response, indent=2)}")
            return {"statusCode": 200, "body": f"Stopped instances: {instance_ids}"}

        # Handle RunInstances action
        elif action == "RunInstances":
            response = ec2_client.run_instances(
                ImageId='ami-0453ec754f44f9a4a',
                InstanceType='t2.micro',
                MinCount=1,
                MaxCount=1,
                KeyName='keypair',
                SecurityGroupIds=['sg-06f61544db7ec64aa'],
                SubnetId='subnet-0cf534bdaccf98326',
                TagSpecifications=[
                    {
                        'ResourceType': 'instance',
                        'Tags': [{'Key': 'Name', 'Value': 'MyAmazonLinuxInstance'}]
                    }
                ]
            )
            instance_ids = [instance['InstanceId'] for instance in response['Instances']]
            logger.info(f"New Amazon Linux instances launched: {instance_ids}")
            return {"statusCode": 200, "body": f"New Amazon Linux instances launched: {instance_ids}"}

        # Handle TerminateInstances action
        elif action == "TerminateInstances":
            response = ec2_client.describe_instances(
                Filters=[{'Name': 'instance-state-name', 'Values': ['running']}]
            )
            instance_ids = [
                instance['InstanceId']
                for reservation in response['Reservations']
                for instance in reservation['Instances']
            ]
            if not instance_ids:
                logger.info("No running instances found.")
                return {"statusCode": 200, "body": "No running instances to terminate."}

            logger.info(f"Terminating instances: {instance_ids}")
            termination_response = ec2_client.terminate_instances(InstanceIds=instance_ids)
            logger.info(f"Termination response: {json.dumps(termination_response, indent=2)}")
            return {"statusCode": 200, "body": f"Terminated instances: {instance_ids}"}

        # Handle DescribeSecurityGroups action
        elif action == "DescribeSecurityGroups":
            response = ec2_client.describe_security_groups()
            security_groups = [
                {"GroupId": sg['GroupId'], "GroupName": sg['GroupName']}
                for sg in response['SecurityGroups']
            ]
            logger.info(f"Security Groups: {security_groups}")
            return {"statusCode": 200, "body": {"SecurityGroups": security_groups}}

        # Handle CreateSecurityGroup action
        elif action == "CreateSecurityGroup":
            response = ec2_client.create_security_group(
                GroupName='NewSecurityGroup',
                Description='Created by Lambda'
            )
            logger.info(f"Created security group: {response['GroupId']}")
            return {"statusCode": 200, "body": f"Created security group: {response['GroupId']}"}

        elif action == "DescribeLaunchTemplates":
            response = ec2_client.describe_launch_templates()
            launch_templates = [
                {
                    "LaunchTemplateId": lt['LaunchTemplateId'],
                    "LaunchTemplateName": lt['LaunchTemplateName'],
                    "DefaultVersionNumber": lt['DefaultVersionNumber'],
                    "LatestVersionNumber": lt['LatestVersionNumber'],
                    "CreatedBy": lt.get('CreatedBy', 'Unknown'),
                }
                for lt in response['LaunchTemplates']
            ]
            logger.info(f"Launch Templates: {json.dumps(launch_templates, indent=2)}")
            return {"statusCode": 200, "body": {"LaunchTemplates": launch_templates}}

        # Handle DeleteSecurityGroup action
        elif action == "DeleteSecurityGroup":
            response = ec2_client.describe_security_groups()
            security_groups = [
                sg['GroupId'] for sg in response['SecurityGroups']
                if sg['GroupName'] != "default"
            ]

            for sg_id in security_groups:
                try:
                    ec2_client.delete_security_group(GroupId=sg_id)
                    logger.info(f"Deleted security group {sg_id}")
                except Exception as e:
                    logger.warning(f"Failed to delete security group {sg_id}: {e}")

            return {"statusCode": 200, "body": f"Deleted security groups: {security_groups}"}

        # Handle RevokeSecurityGroupEgress action
        elif action == "RevokeSecurityGroupEgress":
            response = ec2_client.describe_security_groups()
            security_groups = [sg['GroupId'] for sg in response['SecurityGroups']]

            for sg_id in security_groups:
                try:
                    ec2_client.revoke_security_group_egress(
                        GroupId=sg_id,
                        IpPermissions=[
                            {"IpProtocol": "-1", "IpRanges": [{"CidrIp": "0.0.0.0/0"}]}
                        ]
                    )
                    logger.info(f"Revoked all egress rules for security group {sg_id}")
                except Exception as e:
                    logger.warning(f"Failed to revoke egress rules for {sg_id}: {e}")

            return {"statusCode": 200, "body": f"Revoked egress rules for security groups: {security_groups}"}

        # Handle RevokeSecurityGroupIngress action
        elif action == "RevokeSecurityGroupIngress":
            response = ec2_client.describe_security_groups()
            security_groups = [sg['GroupId'] for sg in response['SecurityGroups']]

            for sg_id in security_groups:
                try:
                    ec2_client.revoke_security_group_ingress(
                        GroupId=sg_id,
                        IpPermissions=[
                            {"IpProtocol": "-1", "IpRanges": [{"CidrIp": "0.0.0.0/0"}]}
                        ]
                    )
                    logger.info(f"Revoked all ingress rules for security group {sg_id}")
                except Exception as e:
                    logger.warning(f"Failed to revoke ingress rules for {sg_id}: {e}")

            return {"statusCode": 200, "body": f"Revoked ingress rules for security groups: {security_groups}"}

        # Handle AuthorizeSecurityGroupIngress action
        elif action == "AuthorizeSecurityGroupIngress":
            response = ec2_client.describe_security_groups()
            security_groups = [sg['GroupId'] for sg in response['SecurityGroups']]

            for sg_id in security_groups:
                try:
                    ec2_client.authorize_security_group_ingress(
                        GroupId=sg_id,
                        IpPermissions=[
                            {"IpProtocol": "tcp", "FromPort": 22, "ToPort": 22, "IpRanges": [{"CidrIp": "0.0.0.0/0"}]}
                        ]
                    )
                    logger.info(f"Authorized SSH ingress for security group {sg_id}")
                except Exception as e:
                    logger.warning(f"Failed to authorize ingress for {sg_id}: {e}")

            return {"statusCode": 200, "body": f"Authorized ingress rules for security groups: {security_groups}"}

        # Handle DescribeInstances action
        elif action == "DescribeInstances":
            response = ec2_client.describe_instances()
            instances = [
                {
                    "InstanceId": instance['InstanceId'],
                    "State": instance['State']['Name']
                }
                for reservation in response['Reservations']
                for instance in reservation['Instances']
            ]
            logger.info(f"Instances: {instances}")
            return {"statusCode": 200, "body": {"Instances": instances}}

        # Handle DescribeSubnets action
        elif action == "DescribeSubnets":
            response = ec2_client.describe_subnets()
            subnets = [{"SubnetId": subnet['SubnetId'], "CidrBlock": subnet['CidrBlock']} for subnet in response['Subnets']]
            logger.info(f"Subnets: {subnets}")
            return {"statusCode": 200, "body": {"Subnets": subnets}}

        # Handle DescribeHosts action
        elif action == "DescribeHosts":
            response = ec2_client.describe_hosts()
            hosts = [{"HostId": host['HostId'], "State": host['State']} for host in response['Hosts']]
            logger.info(f"Hosts: {hosts}")
            return {"statusCode": 200, "body": {"Hosts": hosts}}

        # Handle DescribeVpcs action
        elif action == "DescribeVpcs":
            response = ec2_client.describe_vpcs()
            vpcs = [{"VpcId": vpc['VpcId'], "State": vpc['State']} for vpc in response['Vpcs']]
            logger.info(f"Vpcs: {vpcs}")
            return {"statusCode": 200, "body": {"Vpcs": vpcs}}
                # Handle DescribeAvailabilityZones action
        elif action == "DescribeAvailabilityZones":
            response = ec2_client.describe_availability_zones()
            availability_zones = [{"ZoneName": zone['ZoneName'], "State": zone['State']} for zone in response['AvailabilityZones']]
            logger.info(f"Availability Zones: {availability_zones}")
            return {"statusCode": 200, "body": {"AvailabilityZones": availability_zones}}

        # Handle DescribeKeyPairs action
        elif action == "DescribeKeyPairs":
            response = ec2_client.describe_key_pairs()
            key_pairs = [{"KeyName": key_pair['KeyName'], "KeyPairId": key_pair['KeyPairId']} for key_pair in response['KeyPairs']]
            logger.info(f"Key Pairs: {key_pairs}")
            return {"statusCode": 200, "body": {"KeyPairs": key_pairs}}

        # Handle DescribeImages action
        elif action == "DescribeImages":
            response = ec2_client.describe_images()
            images = [{"ImageId": image['ImageId'], "Name": image['Name']} for image in response['Images']]
            logger.info(f"Images: {images}")
            return {"statusCode": 200, "body": {"Images": images}}

        # Handle DescribeVolumes action
        elif action == "DescribeVolumes":
            response = ec2_client.describe_volumes()
            volumes = [{"VolumeId": volume['VolumeId'], "Size": volume['Size'], "State": volume['State']} for volume in response['Volumes']]
            logger.info(f"Volumes: {volumes}")
            return {"statusCode": 200, "body": {"Volumes": volumes}}

        # Handle DescribeInstanceStatus action
        elif action == "DescribeInstanceStatus":
            response = ec2_client.describe_instance_status()
            instance_status = [
                {
                    "InstanceId": status['InstanceId'],
                    "Status": status['InstanceStatus']['Status']
                }
                for status in response['InstanceStatuses']
            ]
            logger.info(f"Instance Status: {instance_status}")
            return {"statusCode": 200, "body": {"InstanceStatus": instance_status}}

        # Handle DescribePlacementGroups action
        elif action == "DescribePlacementGroups":
            response = ec2_client.describe_placement_groups()
            placement_groups = [
                {"GroupName": pg['GroupName'], "State": pg['State']}
                for pg in response['PlacementGroups']
            ]
            logger.info(f"Placement Groups: {placement_groups}")
            return {"statusCode": 200, "body": {"PlacementGroups": placement_groups}}

        # Handle DescribeInstanceTypes action
        elif action == "DescribeInstanceTypes":
            response = ec2_client.describe_instance_types()
            instance_types = [{"InstanceType": it['InstanceType']} for it in response['InstanceTypes']]
            logger.info(f"Instance Types: {instance_types}")
            return {"statusCode": 200, "body": {"InstanceTypes": instance_types}}

        # Handle DescribeInstanceTypeOfferings action
        elif action == "DescribeInstanceTypeOfferings":
            response = ec2_client.describe_instance_type_offerings()
            offerings = [
                {"InstanceType": offering['InstanceType'], "Location": offering['Location']}
                for offering in response['InstanceTypeOfferings']
            ]
            logger.info(f"Instance Type Offerings: {offerings}")
            return {"statusCode": 200, "body": {"InstanceTypeOfferings": offerings}}

        # Handle DescribeAddresses action
        elif action == "DescribeAddresses":
            response = ec2_client.describe_addresses()
            addresses = [
                {"PublicIp": addr['PublicIp'], "AllocationId": addr['AllocationId']}
                for addr in response['Addresses']
            ]
            logger.info(f"Addresses: {addresses}")
            return {"statusCode": 200, "body": {"Addresses": addresses}}

        # Handle DescribeVolumeStatus action
        elif action == "DescribeVolumeStatus":
            response = ec2_client.describe_volume_status()
            volume_status = [
                {"VolumeId": vs['VolumeId'], "Status": vs['VolumeStatus']}
                for vs in response['VolumeStatuses']
            ]
            logger.info(f"Volume Status: {volume_status}")
            return {"statusCode": 200, "body": {"VolumeStatus": volume_status}}

        # Handle GetEbsEncryptionByDefault action
        elif action == "GetEbsEncryptionByDefault":
            response = ec2_client.get_ebs_encryption_by_default()
            encryption_enabled = response.get('EbsEncryptionByDefault', False)
            logger.info(f"EBS Encryption by Default: {encryption_enabled}")
            return {"statusCode": 200, "body": {"EbsEncryptionByDefault": encryption_enabled}}

        # Handle GetDefaultCreditSpecification action
        elif action == "GetDefaultCreditSpecification":
            try:
                response = ec2_client.describe_instance_types()
                instance_families = {it['InstanceType'].split('.')[0] for it in response['InstanceTypes']}
                burstable_families = {"t2", "t3", "t3a", "t4g"}
                instance_families = instance_families.intersection(burstable_families)

                credit_specifications = {}
                for family in instance_families:
                    try:
                        spec_response = ec2_client.get_default_credit_specification(
                            InstanceFamily=family
                        )
                        credit_specifications[family] = spec_response.get('InstanceFamilyCreditSpecification', {})
                    except Exception as e:
                        logger.warning(f"Failed to get credit spec for family {family}: {e}")

                logger.info(f"Default credit specifications: {credit_specifications}")
                return {"statusCode": 200, "body": {"CreditSpecifications": credit_specifications}}

            except Exception as e:
                logger.error(f"Error in GetDefaultCreditSpecification: {str(e)}")
                return {"statusCode": 500, "body": f"Error: {str(e)}"}

        elif action == "GetInstanceMetadataDefaults":
            # You can retrieve EC2 metadata about instance defaults using DescribeInstances
            response = ec2_client.describe_instance_types()

            instance_types = [
                {
                    "InstanceType": it['InstanceType'],
                    "VCpuInfo": it['VCpuInfo'],
                    "MemoryInfo": it['MemoryInfo'],
                    "InstanceStorageInfo": it.get('InstanceStorageInfo', 'No storage info'),
                    "NetworkInfo": it['NetworkInfo'],
                }
                for it in response['InstanceTypes']
            ]

            logger.info(f"Default Instance Metadata: {json.dumps(instance_types, indent=2)}")
            return {"statusCode": 200, "body": json.dumps({"InstanceTypes": instance_types})}

        # Catch-all for unsupported actions
        # Catch-all for unsupported actions
        else:
            logger.warning(f"Unsupported action: {action}. Returning success response.")
            return {
                "statusCode": 200,
                "body": json.dumps({
                    "message": f"Unsupported action: {action}. No action performed.",
                    "eventName": action,
                }),
            }

    except Exception as e:
        logger.error(f"Error processing EC2 action: {str(e)}")
        return {"statusCode": 500, "body": f"Error: {str(e)}"}

